In [ ]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [ ]:
import httpx

from urllib.parse import urljoin

from will_it_saturate.server import (
    FastAPIUvicornServer,
    NginxDockerServer,
    DjangoGunicornWSGIServer,
)
from will_it_saturate.core import FilesystemCreator, Epoch, Result, BenchmarkFile
from will_it_saturate.client import HttpxClient, AioHttpClient, WrkClient
from will_it_saturate.persistence import SqliteRepository

<IPython.core.display.Javascript object>

# Remote Benchmark

In [ ]:
def run_epoch(server, client, epoch):
    control_base = "http://localhost:8001/"
    control_epochs = urljoin(control_base, "epochs/")
    r = httpx.post(control_epochs, json=epoch.dict_without_file_creator())

    for bfile in r.json()["files"]:
        kwargs = {k: v for k, v in bfile.items() if k != "creator"}
        epoch.files.append(BenchmarkFile(**kwargs))

    control_servers = urljoin(control_base, "servers/")
    r = httpx.post(control_servers, json=server.dict())
    server = server.__class__(**r.json())

    epoch.urls = [server.file_to_url(file) for file in epoch.files]

    result = Result(
        server=server.name,
        client=client.name,
        file_size=epoch.file_size,
        complete_size=epoch.complete_size,
    )

    result.elapsed = client.measure(epoch)
    return result

<IPython.core.display.Javascript object>

In [ ]:
byte, file_size, duration = 8, 10 ** 7, 3
epoch = Epoch(file_size=file_size, duration=duration)

<IPython.core.display.Javascript object>

In [ ]:
server = FastAPIUvicornServer()
client = HttpxClient(name="httpx")

<IPython.core.display.Javascript object>

In [ ]:
%%time
result = run_epoch(server, client, epoch)

measure
CPU times: user 28.1 ms, sys: 10.2 ms, total: 38.3 ms
Wall time: 9.38 s


<IPython.core.display.Javascript object>

In [ ]:
result.bytes_per_second

59206337.146475695

<IPython.core.display.Javascript object>

In [ ]:
%%time
result = run_epoch(10 ** 7)

measure
CPU times: user 38.6 ms, sys: 11.3 ms, total: 49.9 ms
Wall time: 9.41 s


<IPython.core.display.Javascript object>

In [ ]:
%%time
result = run_epoch(10 ** 6)

measure
CPU times: user 28.8 ms, sys: 10.4 ms, total: 39.1 ms
Wall time: 9.7 s


<IPython.core.display.Javascript object>

In [ ]:
result.bytes_per_second

56193971.8565544

<IPython.core.display.Javascript object>

In [ ]:
%%time

byte = 8
gigabit = 10 ** 9
bandwidth = gigabit / byte
duration = 3
file_size = 10 ** 7

epoch = Epoch(file_size=file_size, duration=duration)

control_base = "http://localhost:8001/"
control_epochs = urljoin(control_base, "epochs/")

r = httpx.post(control_epochs, json=epoch.dict_without_file_creator())

for bfile in r.json()["files"]:
    kwargs = {k: v for k, v in bfile.items() if k != "creator"}
    epoch.files.append(BenchmarkFile(**kwargs))


server = FastAPIUvicornServer()
control_servers = urljoin(control_base, "servers/")
r = httpx.post(control_servers, json=server.dict())
server = FastAPIUvicornServer(**r.json())

epoch.urls = [server.file_to_url(file) for file in epoch.files]

client = HttpxClient(name="httpx")


result = Result(
    server=server.name,
    client=client.name,
    file_size=epoch.file_size,
    complete_size=epoch.complete_size,
)

result.elapsed = client.measure(epoch)
result

measure
CPU times: user 31.8 ms, sys: 10.2 ms, total: 42.1 ms
Wall time: 9.3 s


Result(server='fastAPI/uvicorn', client='httpx', file_size=10000000, elapsed=6.217392495, complete_size=375000000)

<IPython.core.display.Javascript object>

In [ ]:
result.bytes_per_second

60314673.7642144

<IPython.core.display.Javascript object>

# Local Benchmark

In [ ]:
# dont_test

byte = 8
gigabit = 10 ** 9
bandwidth = gigabit / byte
duration = 3
file_size = 10 ** 7
file_creator = FilesystemCreator()


# server = DjangoGunicornWSGIServer()
server = FastAPIUvicornServer()
# client = AioHttpClient(name="aiohttp")
client = HttpxClient(name="httpx")
# repository = SqliteRepository.build_repository("foo.db")

epoch = Epoch(file_size=file_size, duration=duration, file_creator=file_creator)

epoch.create_files()

result = BenchmarkResult(
    server=server.name,
    client=client.name,
    file_size=epoch.file_size,
    complete_size=epoch.complete_size,
)

if not server.started:
    server.start

result.elapsed = client.measure(epoch)

<IPython.core.display.Javascript object>